## Replace Manual version of Logistic Regression with TF based version.

In [1]:
import nltk
from nltk.corpus import twitter_samples
import pandas as pd
import tensorflow as tf

In [2]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

process_tweet(): cleans the text, tokenizes it into separate words, removes stopwords, and converts words to stems.

In [4]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets

    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        # 1 remove stopwords
        if word in stopwords_english:
            continue
        # 2 remove punctuation
        if word in string.punctuation:
            continue
        # 3 stemming word
        word = stemmer.stem(word)
        # 4 Add it to tweets_clean
        tweets_clean.append(word)

    return tweets_clean

build_freqs counts how often a word in the 'corpus' (the entire set of tweets) was associated with a positive label '1' or a negative label '0.
Then builds the freqs dictionary, where each key is a (word,label) tuple and the value is the count of its frequency within the corpus of tweets.

In [5]:
def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}

    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)

            #Update the count of pair if present, set it to 1 otherwise
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

select the set of positive and negative tweets

In [6]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

split the data into two pieces, one for training and one for testing

In [7]:
from sklearn.model_selection import train_test_split

train_pos, test_pos = train_test_split(all_positive_tweets, test_size=0.3)
train_neg, test_neg = train_test_split(all_negative_tweets, test_size=0.3)

train_x = train_pos + train_neg
test_x = test_pos + test_neg

combine positive and negative labels

In [8]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

create frequency dictionary

In [9]:
freqs = build_freqs(train_x, train_y)

In [10]:
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 10391


Taking an example

In [11]:
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 @emily_etc I always walk past this place! May have to go in and try them for myself now ! :D

This is an example of the processed version of the tweet: 
 ['alway', 'walk', 'past', 'place', 'may', 'go', 'tri', ':D']


In [12]:
def extract_features(tweet, freqs):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''
    # tokenizes, stems, and removes stopwords
    output = []
    for word_l in tweet:
        word_l = process_tweet(word_l)

        # 3 elements in the form of a 1 x 3 vector
        x = np.zeros((1, 3))

        #bias term is set to 1
        x[0,0] = 1

        # loop through each word in the list of words
        for word in word_l:

            # increment the word count for the positive label 1
            x[0,1] += freqs.get((word, 1.0),0)

            # increment the word count for the negative label 0
            x[0,2] += freqs.get((word, 0.0),0)


        assert(x.shape == (1, 3))
        output.append(x)
    return output

In [13]:
final_model = tf.keras.models.Sequential([ tf.keras.layers.Dense(2, activation=tf.nn.softmax) ])

In [14]:
final_model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_model.fit(tf.convert_to_tensor(extract_features(train_x, freqs)), train_y, epochs=1000)

Epoch 1/1000
219/219 [==============================] - 1s 1ms/step - loss: 24.4524 - accuracy: 0.9903
Epoch 2/1000
219/219 [==============================] - 0s 1ms/step - loss: 9.3728 - accuracy: 0.9943
Epoch 3/1000
219/219 [==============================] - 0s 1ms/step - loss: 8.9891 - accuracy: 0.9944
Epoch 4/1000
219/219 [==============================] - 0s 1ms/step - loss: 8.5091 - accuracy: 0.9944
Epoch 5/1000
219/219 [==============================] - 0s 1ms/step - loss: 8.4425 - accuracy: 0.9946
Epoch 6/1000
219/219 [==============================] - 0s 1ms/step - loss: 8.0786 - accuracy: 0.9947
Epoch 7/1000
219/219 [==============================] - 0s 1ms/step - loss: 7.7749 - accuracy: 0.9941
Epoch 8/1000
219/219 [==============================] - 0s 1ms/step - loss: 7.5036 - accuracy: 0.9946
Epoch 9/1000
219/219 [==============================] - 0s 1ms/step - loss: 7.0862 - accuracy: 0.9943
Epoch 10/1000
219/219 [==============================] - 0s 1ms/step - loss: 6.94

In [ ]:
final_model.evaluate(tf.convert_to_tensor(extract_features(test_x, freqs)), test_y)